In [1]:
import os, sys
import collections
# sys.path.append(os.path.join(os.path.dirname(__file__), '..', '..'))

from datetime import datetime
from dateutil.tz import tzlocal
import pytz
import re
import numpy as np
import json
import pandas as pd
import pathlib

# from imaging import analysis, preprocess, reference, equipment, tracking, analysis_param, sessions, animal, file, tif
import pynwb
from pynwb import NWBFile, NWBHDF5IO
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from pynwb.image import ImageSeries
from pynwb.ophys import TwoPhotonSeries, OpticalChannel, ImageSegmentation, \
    Fluorescence, CorrectedImageStack
import scanreader

In [2]:
# ============================== SET CONSTANTS ==========================================
zero_zero_time = datetime.strptime('00:00:00', '%H:%M:%S').time()  # no precise time available
institution = 'DataJoint - testing CorrectedImageStack'

In [3]:
pwd

'/Users/geetikasingh/Desktop/Projects-DJ/Moser/example_nwb'

In [5]:
tiff_file = scanreader.read_scan('./k53_20160530_RSM_125um_41mW_zoom2p2_00001_00001.tif')

In [9]:
tiff_file

In [ ]:
def get_nwb_subject(session_key):
    subj_key = {'animal_id': 'abc123', 'datasource_id': 0}
    subj = {'animal_id': 'abc123',
            'datasource_id': 0,
            'animal_species': 'mouse',
            'animal_name': '123',
            'animal_sex': 'M',
            'animal_dob': datetime.date(2022, 3, 1),
            'color': 'Unknown',
            'animal_notes': 'Sample animal'}
    strain = 'C +/- x GC+/-'

    return pynwb.file.Subject(
        subject_id=f'{subj_key["animal_id"]}-{subj_key["datasource_id"]}-{session_key}',
        description=f'animal_name: {subj["animal_name"]}; color: {subj["color"]}; strain: {strain}; animal_notes: {subj["animal_notes"]}',
        genotype=' x ',
        sex=subj['animal_sex'],
        species=subj['animal_species'],
        date_of_birth=datetime.combine(subj['animal_dob'], zero_zero_time) if subj['animal_dob'] else None)


def export_to_nwb(session_key, output_dir='./', overwrite=True):

    print(f'Exporting to NWB 2.0 for session: {session_key}...')
    # ===============================================================================
    # ============================== META INFORMATION ===============================
    # ===============================================================================

    # -- NWB file - a NWB2.0 file for each session

    session = {'session_name': 'abc123d4fcbb',
                'recording_order': 0,
                'recording_name': 'rrrrd24',
                'animal_id': 'abc123',
                'datasource_id': 0,
                'animal_name': '123',
                'timestamp': datetime.datetime(2022, 3, 1, 5, 30, 10),
                'combined': 'yes',
                'timeseries_name': '123_20220301_ML-100_DJ01_3Openfiled',
                'equipment_type': '2Pminiscope_A',
                'username': 'testuser'}
    file_name = '_'.join(
        [session['session_name'],
         session['timestamp'].strftime('%Y-%m-%d'),
         str(session['timeseries_name'])])
    nwbfile = NWBFile(
            identifier=file_name,
            #TODO: Do we store publications anywhere in the database?
            related_publications='',
            experiment_description='',
            session_description='Imaging session',
            session_start_time=datetime.combine(session['timestamp'], zero_zero_time),
            file_create_date=datetime.now(tzlocal()),
            experimenter='Test',
            institution=institution,
            #TODO: Fetch keywords from database
            keywords=['Two-photon imaging'])

    # ==========================================================================
    # ----- Create a Device, create an OpticalChannel and an ImagingPlane ------
    # ==========================================================================
    system_names = np.array(['System v1.0'])
    scope_names = np.array(['Scope 1'])
    device_name = ["{}_{}".format(system_name_, scope_name_) for system_name_, scope_name_ in zip(system_names,scope_names)][0]
    device = nwbfile.create_device(name=device_name,
                                   description="",
                                   manufacturer=""
                                   )

    #TODO: Check optical channel
    optical_channel = OpticalChannel(name="OpticalChannel",
                                     description="an optical channel",
                                     emission_lambda=500.
                                     )

    #TODO: Frame rate or volume rate == imaging rate?  Frame rate : 2D, volume rate : 3D (what to choose?)
    nwbfile.subject = get_nwb_subject(session_key)
    num_planes = tiff_file.num_scanning_depths
    framerate = tiff_file.fps
    # volume_rate = tiff_file.volume_rate
    plane_keys = [{'session_name': 'abc123d4fcbb',
                    'recording_order': 0,
                    'recording_name': 'rrrrd24',
                    'dataset_name': 'dddd280d',
                    'center_plane': 0},
                    {'session_name': 'abc123d4fcbb',
                    'recording_order': 0,
                    'recording_name': 'rrrrd24',
                    'dataset_name': 'dddd280d',
                    'center_plane': 1}]
    ophys_module = nwbfile.create_processing_module(name='ophys',
                                                    description='optical physiology processed data'
                                                    )
    img_seg = ImageSegmentation()
    corrected_image_stacks = []
    roi_resp_series = []
    for plane in plane_keys:
        imaging_plane = nwbfile.create_imaging_plane(name="ImagingPlane"+ '_' + str(plane['center_plane']),
                                                    optical_channel=optical_channel,
                                                    imaging_rate=framerate[0],
                                                    description="",
                                                    device=device,
                                                    excitation_lambda=600.,
                                                    indicator="GFP",
                                                    location="",
                                                    grid_spacing=(),
                                                    grid_spacing_unit="",
                                                    origin_coords=(),
                                                    origin_coords_unit=""
                                                    )

        # ==========================================================================
        # ---------------------------- Insert data file ----------------------------
        # ==========================================================================

        #TODO: Should the files be combined here?
        file_path = (sessions.Recording.Data * file.PhysicalFile & (sessions.Session & plane) & 'filetype="TIFRaw"').fetch('file_path')
        image_series = TwoPhotonSeries(name='TwoPhotonSeries2'+'_'+str(plane['center_plane']),
                                        dimension=[100, 100],
                                        external_file=file_path,
                                        imaging_plane=imaging_plane,
                                        starting_frame=[0],
                                        format='external',
                                        starting_time=0.0,
                                        rate=1.0
                                        )
        nwbfile.add_acquisition(image_series)

        # ==========================================================================
        # --------------------- MotionCorrection information -----------------------
        # ==========================================================================

        mean_image_enhanced_corr = (analysis.ProjectionCorr & session_key & plane).fetch1('mean_image_enhanced_corr')
        height, width = (mean_image_enhanced_corr.shape)
        mean_image_enhanced_corr = np.reshape(mean_image_enhanced_corr, (-1, height, width))
        #TODO: mean_image_enhanced_corr is of the shape [frame, x, y] or [frame, x, y, z], 
        # adding (num_frames_scanimage from tif.Tif.SI, shoulud I use num_frames from tif.Tif?)
        num_frames_scanimage = (tif.Tif.SI & session_key).fetch1('num_frames_scanimage') 
        # mean_image_data =  np.array([num_frames_scanimage] + [mean_image_enhanced_corr])
        corrected = ImageSeries(name='corrected',  # this must be named "corrected"
                                data=mean_image_enhanced_corr,
                                unit='na',
                                format='Average projection of motion corrected stack - contrast enhanced (unwarped)', #TODO: Check format
                                starting_time=0.0,
                                rate=1.0
                                )

        #TODO: Or if else condition could be using rigid non-rigid flag using info from ImagingAnalysis.suite2p
        if (analysis.Projection.Motion & session_key & plane).fetch1('x_mocor') is None:
            #TODO: Check the shape of the block_mocor_x
            block_mocor_x, block_mocor_y = (analysis.Projection.Motion & session_key & plane).fetch1('block_mocor_x', 'block_mocor_y')
            xy_translation_data = list(zip(*block_mocor_x, *block_mocor_y))
        else:
            #TODO: Confirm the shape of the x_mocor, y_mocor : should it be tuple of x,y : check timeseries nwb object
            x_mocor, y_mocor = (analysis.Projection.Motion & session_key & plane).fetch1('x_mocor', 'y_mocor')
            xy_translation_data = zip(x_mocor, y_mocor)

        xy_translation = TimeSeries(name='xy_translation',
                                    data=list(xy_translation_data),
                                    unit='pixels',
                                    starting_time=0.0,
                                    rate=1.0,
                                    )

        corrected_image_stack = CorrectedImageStack(corrected=corrected,
                                                    original=image_series,
                                                    xy_translation=xy_translation,
                                                    )

        corrected_image_stacks.append(corrected_image_stack)